In [11]:
import os
import pandas as pd
from IPython.display import display
from transformers import AutoTokenizer, AutoModelForTokenClassification
from torch import nn
import torch

# Spécifiez le chemin du modèle DistilBERT
distilbert_model_path = r'C:\Users\vikne\Documents\Master 2\Semestre 9\Intelligence artificielle\Travel-Order-Resolver\ai\nlp\models\tokens_classification\distilbert'

# Charger le tokenizer et le modèle DistilBERT
try:
    tokenizer = AutoTokenizer.from_pretrained(distilbert_model_path)
    model = AutoModelForTokenClassification.from_pretrained(distilbert_model_path)
    print("Modèle DistilBERT et tokenizer chargés avec succès.")
except Exception as e:
    print(f"Erreur lors du chargement du modèle DistilBERT ou du tokenizer: {e}")

# Liste des phrases pour l'évaluation
phrases = [
    "Je pars de Paris et j'arrive à Marseille.",
    "Je vais à Bordeaux en partant de Toulouse.",
    "Mon trajet va de VILLIERS SUR LOIR à JARNY.",
    "Nous partons de Lyon et nous arrivons à Nice.",
    "Le départ est prévu à Lille, et l'arrivée se fera à Strasbourg.",
    "Mon itinéraire inclut un départ de Montpellier et une arrivée à Perpignan.",
    "Le train partira de Nantes pour arriver à Rennes.",
    "Un vol de Genève à Zurich est prévu demain matin.",
    "Je vais de Grenoble à Dijon en voiture.",
    "Je voyage de Bruxelles à Amsterdam ce week-end.",
    "L'itinéraire commence à Marseille et se termine à Toulon.",
    "Nous prenons un train de Rouen pour aller au Havre.",
    "Le départ sera à Limoges et l'arrivée à Poitiers.",
    "Je quitte Monaco pour aller à Menton.",
    "Le bus va de Caen à Cherbourg.",
    "Il part de Bayonne et arrive à Biarritz.",
    "Je vais de Chamonix à Annecy.",
    "Nous roulons de Bastia à Ajaccio.",
    "Le ferry part de Calais pour arriver à Douvres.",
    "Je pars de Bordeaux et j'arrive à Paris.",
    "Notre vol va de Madrid à Barcelone.",
    "Un trajet en voiture de Milan à Rome est prévu.",
    "Je vais à Paris en partant de Lyon."
]

# Liste pour stocker les résultats
results = []

# Fonction pour afficher les prédictions avec DistilBERT
for phrase in phrases:
    try:
        # Tokenizer la phrase
        inputs = tokenizer(phrase, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            # Passer les tokens dans le modèle
            outputs = model(**inputs)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=2)

        # Obtenir les étiquettes prédits pour chaque token
        tokens = tokenizer.tokenize(tokenizer.decode(inputs['input_ids'][0]))  # Récupérer les tokens
        labels = [model.config.id2label[pred.item()] for pred in predictions[0]]

        # Ajouter les entités détectées à la liste des résultats
        for token, label in zip(tokens, labels):
            if label != 'O':  # On ignore les tokens avec le label 'O'
                results.append({"Phrase": phrase, "Entity": token, "Label": label})

    except Exception as e:
        print(f"Erreur lors du traitement de la phrase '{phrase}': {e}")

# Convertir les résultats en DataFrame
df = pd.DataFrame(results)

# Afficher le tableau
if not df.empty:
    display(df)
else:
    print("Aucune entité détectée.")

Modèle DistilBERT et tokenizer chargés avec succès.


,Phrase,Entity,Label
0,Je pars de Paris et j'arrive à Marseille.,paris,B-DEP
1,Je pars de Paris et j'arrive à Marseille.,marseille,B-ARR
2,Je vais à Bordeaux en partant de Toulouse.,bordeaux,B-ARR
3,Je vais à Bordeaux en partant de Toulouse.,toulouse,B-DEP
4,Je vais à Bordeaux en partant de Toulouse.,[SEP],B-DEP
5,Mon trajet va de VILLIERS SUR LOIR à JARNY.,villiers,B-DEP
6,Mon trajet va de VILLIERS SUR LOIR à JARNY.,sur,I-DEP
7,Mon trajet va de VILLIERS SUR LOIR à JARNY.,lo,I-DEP
8,Mon trajet va de VILLIERS SUR LOIR à JARNY.,##ir,I-DEP
9,Mon trajet va de VILLIERS SUR LOIR à JARNY.,jar,B-ARR
